In [ ]:
# Funciones UNA POR UNA:





In [1]:
# 0. CARGAMOS LAS LIBRERIAS

In [1]:
import pickle
import numpy as np
#from numpy import *
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt
import numpy as np

#loss='binary_crossentropy'

Using TensorFlow backend.


# 1. Preparacion de matrices aumentadas y de matrices test

Cada una de las matrices, que originalmente se crearon sobre 13.5 km por lado con 27x27 pixels representando 500m (para poder aplicar la tecnica stir de data augmentation), ya ha convertido en 9 matrices de 25x25 que alimentarà la cnn.
Estas matrices estan agrupadas por años y guardadas en archivos pickle.

En una primera version la red neuronal se entreno y testeó con estos datos, pero resulto en aciertos del 98% que hicieron pensar en que el stir no evitaba que la red reconociera imagenes casi identicas.

En principio se creo un sistema para recoger 1 de cada 5 imagenes originaes (teniendo en cuenta que cada imagen aparece 9 veces seguidas en el listado anual), pero seguiamos teniendo un nivel de acierto demasiado alto por lo que optamos por asegurarnos que la red no habia visto previamente los puntos del test usando el 2019 nueve como anyo de test, sin que sus datos hayan tenido ningun contacto con el entrenamiento. (entendiendo que en la seleccion de training-test, es mas posible que haya confusiones o filtraciones, especialmente al tratarse en este caso de una seleccion de los puntos aumentados. 



## 1.FUNCION SepararNoAumentados(anyo,AUMENTAR)


In [2]:
## 1.FUNCION SepararNoAumentados(anyo,AUMENTAR)



# ANYO A ANALIZAR Y CADA CUANTOS DEJAR UNA MATRIZ SIN AUMENTAR (PARA VALIDAR)
#ORIGINALMENTE RECOGE LOS MULTIPLOS DEE 5, UN 20%)
#SI RECOGE TODOS LOS MULTIPLOS DE 1, UN 100% SIN DATA AUGMENTATION)
#SI RECOGE TODOS LOS MULTIPLOS DE 10000, UN 0% SIN DATA AUGMENTATION: RESERVAR UN ANYO PARA LA VALIDACION)


def SepararNoAumentados(anyo,AUMENTAR=5):

    #anyo=2019

    #para abrirlos
    direccion=('DATOSLISTOS25X25/m_NoF4cnn/mSIN_F_25_'+str(anyo)+'.pkl')
    with open(direccion, 'rb') as f:
        SinFuego2019 = pickle.load(f)   
        
    print(f'Del anyo {anyo} entran {len(SinFuego2019)} sin fuego')

    
    direccion=('DATOSLISTOS25X25/m_F4cnn/mCON_F_25_'+str(anyo)+'.pkl')
    with open(direccion, 'rb') as f:
        ConFuego2019 = pickle.load(f)  
        
    print(f'Del anyo {anyo} entran {len(ConFuego2019)} Con fuego')



    # puesto qeu cada foto son 9 identificamos la posicion de lprimera de cada remesa
    listaImgOriginales = []
    contador=0
    while contador < (len(ConFuego2019)):
        if contador% 9 == 0:
            listaImgOriginales.append(contador)
        contador+=1

    # para seleccionar un 20% de fotos para el test (0.2) recogemos las posicion inicial de cada 5 fotos 
    posicionInicioMTest=[]
    contador=0
    for posicionMatrix in listaImgOriginales:
##AQUI SE PUEDE DECIDIR EL PORCENTAGE (ORIGINALMENTE RECOGE LOS MULTIPLOS DEE 5, UN 20%)
        if contador% AUMENTAR == 0:
            posicionInicioMTest.append(posicionMatrix)
        contador+=1
        
    print(f'Se seleccionan para NO SER AUMENTADOS: {len(posicionInicioMTest)} de {len(listaImgOriginales)}')
    

    
    # de cada posicion inicial necesitareemos ELIMINAR las 8 siguientes (que son la misma)
    #posicionesAeliminar=[]
    #for x in posicionInicioMTest:
     #   for n in range(8):
      #      posicionesAeliminar.append(x+1+n) 

    
    # con el listado de las posicions separamos las imagenes para test de las para train
    Validacion_F=[]
    train_F=[]
    Validacion_NF=[]
    train_NF=[]
    
    
    
    #anyadirmoos un numero para que posteriormente lo pueda buscar
    posicionInicioMTest.append(100000)
    
    contadorTest=0
    contadorTrain=0
    while contadorTrain<(len(ConFuego2019)):

        #print(contadorTrain,contadorTest)
        
        if contadorTrain==posicionInicioMTest[contadorTest]:
            #recogemos la imagen central con 0 stir
            Validacion_F.append(ConFuego2019[contadorTest+4])
            Validacion_NF.append(SinFuego2019[contadorTest+4])
            contadorTest+=1
            contadorTrain+=9
        else:
            train_F.append(ConFuego2019[contadorTrain])
            train_NF.append(SinFuego2019[contadorTrain])
            contadorTrain+=1

    print(len(train_F),len(Validacion_F),len(train_NF),len(Validacion_NF))
    return train_F,train_NF,Validacion_F,Validacion_NF








#train_F,train_NF,test_F,test_NF=SepararNoAumentados(2000,AUMENTAR=5)

In [3]:
## 2.Funcion prepararDAtos(matricesValidacionFuego,matricesValidacionSINFuego)



def prepararDAtos(matricesValidacionFuego,matricesValidacionSINFuego):
    ValidacionX=matricesValidacionSINFuego+matricesValidacionFuego
    FUEGO=[1]
    NoF=[0]
    ValidacionY=(NoF*len(matricesValidacionSINFuego))+(FUEGO*len(matricesValidacionFuego))
    
    #print(ValidacionY)
    #del total hasta la mitad son fuegos y la otra mitad son no fuegos

    #preparamos el conjunto para entrarlos en la ccn entrenada para predecir su salida:

    #ELIMINANDO LOS NAS    
    Vx_base = np.array(ValidacionX)

    nas=0
    for lista in range(len(Vx_base)):
        for matriz in range(8):
            where_are_NaNs = np.isnan(Vx_base[lista][matriz])
            Vx_base[lista][matriz][where_are_NaNs] = 0
            nas+=1
    print(f'Se han detectado {nas} valores nulos')

    #DANDOLE EL FORMATO FINAL A X
    try:
        Vx_base=Vx_base.transpose (0,2,3,1)
        #Vx_base.shape


    #DANDOLE EL FORMATO FINAL A Y
        Vy_base = np.array(ValidacionY)
        #print(Vy_base)
        Vy_base= to_categorical(Vy_base)
        
        #print(Vy_base)
        print(f'Esta funcion devuelve: una lista de {len(ValidacionX)} matrices X de dimension {len(ValidacionX[0])}')
        print(f'y una lista de {len(Vy_base)} etiquetas Y. La primera mitad sin fuego estan represntados con: {(Vy_base[0])} y la segunda mitad con fuego con {(Vy_base[-1])}')
        return Vx_base,Vy_base    
    
    except:
        print('Vx_base,Vy_base,esta vacio') 
        Vx_base=1
        Vy_base=1
        return Vx_base,Vy_base 


#x_base,y_base=prepararDAtos(train_F,train_NF)
#x_validacion,y_validacion=prepararDAtos(test_F,test_NF)
#len(x_base),len(y_base),len(x_validacion),len(y_validacion)

In [4]:
#3. funcion datosTrainingValidationXanyo(anyo,AUMENTAR)
#Combina las 2 anteriores
#anyo=2019

def datosTrainingValidationXanyo(anyo,AUMENTAR):
    
    train_F,train_NF,matricesValidacionFuego,matricesValidacionSINFuego=SepararNoAumentados(anyo,AUMENTAR)
    #print(f'Los datos aumentados para el training son {len(train_F)} de fuego y {len(train_NF)} de Nofuego')

    #print(f'Los datos NO aumentados para la validacion son {len(matricesValidacionFuego)} de fuego y {len(matricesValidacionSINFuego)} de Nofuego')

    Vx_base,Vy_base=prepararDAtos(matricesValidacionFuego,matricesValidacionSINFuego)

    x_base,y_base=prepararDAtos(train_F,train_NF)

    #hasta aqui tenemos X UN ANYO!!!
    #2 listadoS con fuego y no fuegos ORDENADOS (PRIMERO LOS NOFUEGOS Y DESPUES LOS FUEGOS) 
    #El listado de validacion SIN AUMENTATION esta formado por uno de cada 5 fuegos a lo largo del anyo
    #El listado de training AUMENTADOS esta formado por uno de cada 4 fuegos a lo largo del anyo X 9 VERSIONES 

    #AL estar ordenados hemos dado a la primera mitad el valor 0 y a la segunda el valor 1

    return Vx_base,Vy_base,x_base, y_base

#x_validacion,y_validacion,x_base, y_base=datosTrainingValidationXanyo(2000,5)
#len(x_base),len(y_base),len(x_validacion),len(y_validacion)

In [5]:
#anyo=2019

def juntarTodosLosAnyosTxTyVxVY(anyoinicio,anyofinal,AUMENTAR):

    Vx_base,Vy_base,x_base, y_base = datosTrainingValidationXanyo(anyoinicio,AUMENTAR)
    print(f"________"+str(anyoinicio)+"___________")
    print(anyoinicio)
    #print(Vx_base.shape,Vy_base.shape,x_base.shape, y_base.shape)
    
    for anyo in range((anyoinicio+1),anyofinal):
        Vx_base1,Vy_base1,x_base1, y_base1 = datosTrainingValidationXanyo(anyo,AUMENTAR)
        #print(anyo)
        #print(Vx_base1.shape,Vy_base1.shape,x_base1.shape, y_base1.shape)

        Vx_base=np.append(Vx_base,Vx_base1,axis=0)
        Vy_base=np.append(Vy_base,Vy_base1,axis=0)
        try:
            x_base=np.append(x_base,x_base1,axis=0)
            y_base=np.append(y_base,y_base1,axis=0)
        except:
            x_base=[[1],[1]]
            y_base=[[1],[1]]

        print(anyo)
        #print(Vx_base.shape,Vy_base.shape,x_base.shape, y_base.shape)
    return Vx_base,Vy_base,x_base, y_base

#x_validacion,y_validacion,x_base, y_base=juntarTodosLosAnyosTxTyVxVY(2000,2020,5)

#len(x_base),len(y_base),len(x_validacion),len(y_validacion)

# AJUSTES FINALES Y SELECCION DE LA MEJOR CNN 

In [6]:
# recogemos los datos de 2000 a 2019 guardando para el test 1 de cada 5 matrices sin aumentar

In [7]:
x_test, y_test,x_train,y_train=juntarTodosLosAnyosTxTyVxVY(2000,2019,5)

Del anyo 2000 entran 252 sin fuego
Del anyo 2000 entran 252 Con fuego
Se seleccionan para NO SER AUMENTADOS: 6 de 28
198 6 198 6
Se han detectado 96 valores nulos
Esta funcion devuelve: una lista de 12 matrices X de dimension 8
y una lista de 12 etiquetas Y. La primera mitad sin fuego estan represntados con: [1. 0.] y la segunda mitad con fuego con [0. 1.]
Se han detectado 3168 valores nulos
Esta funcion devuelve: una lista de 396 matrices X de dimension 8
y una lista de 396 etiquetas Y. La primera mitad sin fuego estan represntados con: [1. 0.] y la segunda mitad con fuego con [0. 1.]
________2000___________
2000
Del anyo 2001 entran 207 sin fuego
Del anyo 2001 entran 207 Con fuego
Se seleccionan para NO SER AUMENTADOS: 5 de 23
162 5 162 5
Se han detectado 80 valores nulos
Esta funcion devuelve: una lista de 10 matrices X de dimension 8
y una lista de 10 etiquetas Y. La primera mitad sin fuego estan represntados con: [1. 0.] y la segunda mitad con fuego con [0. 1.]
Se han detectado 25

2013
Del anyo 2014 entran 342 sin fuego
Del anyo 2014 entran 342 Con fuego
Se seleccionan para NO SER AUMENTADOS: 8 de 38
270 8 270 8
Se han detectado 128 valores nulos
Esta funcion devuelve: una lista de 16 matrices X de dimension 8
y una lista de 16 etiquetas Y. La primera mitad sin fuego estan represntados con: [1. 0.] y la segunda mitad con fuego con [0. 1.]
Se han detectado 4320 valores nulos
Esta funcion devuelve: una lista de 540 matrices X de dimension 8
y una lista de 540 etiquetas Y. La primera mitad sin fuego estan represntados con: [1. 0.] y la segunda mitad con fuego con [0. 1.]
2014
Del anyo 2015 entran 153 sin fuego
Del anyo 2015 entran 153 Con fuego
Se seleccionan para NO SER AUMENTADOS: 4 de 17
117 4 117 4
Se han detectado 64 valores nulos
Esta funcion devuelve: una lista de 8 matrices X de dimension 8
y una lista de 8 etiquetas Y. La primera mitad sin fuego estan represntados con: [1. 0.] y la segunda mitad con fuego con [0. 1.]
Se han detectado 1872 valores nulos
Est

In [8]:
len(x_train),len(y_train),len(x_test),len(y_test)

(12042, 12042, 354, 354)

In [9]:
#2019 SIN AUMENTAR como validacion 

x_en2019,y_en2019,x_base, y_base=juntarTodosLosAnyosTxTyVxVY(2019,2020,1)
len(x_en2019)

Del anyo 2019 entran 2205 sin fuego
Del anyo 2019 entran 2205 Con fuego
Se seleccionan para NO SER AUMENTADOS: 245 de 245
0 245 0 245
Se han detectado 3920 valores nulos
Esta funcion devuelve: una lista de 490 matrices X de dimension 8
y una lista de 490 etiquetas Y. La primera mitad sin fuego estan represntados con: [1. 0.] y la segunda mitad con fuego con [0. 1.]
Se han detectado 0 valores nulos
Vx_base,Vy_base,esta vacio
________2019___________
2019


490

In [10]:
len(x_en2019),len(y_en2019)

(490, 490)

In [11]:
#con SOFTMAX y BINARIA    
def elegirCNN(m1MP=0,m2MP=0,mo3D128=0):

    model = Sequential()

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=[25, 25, 8]))
    if m1MP==1:
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    #eliminado en version 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    if m2MP==1:
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    #eliminado en version 2
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    if mo3D128==1:
        model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))

    model.add(Dense(2, activation='softmax'))


    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

    return model



In [20]:
#Definitiva

import subprocess

batch_size=[50,100,300] 
anterior=0.79
limites=[0.90,0.96,0.98]

resultados=[]
identificador=0
for m2 in range(2):
    for veces in range(3):
        print(f'vuelta{veces}')
        numeroEpocas=10
        epocaMejorScore=0
                    
    #blokeamos los que han dado mejor resultado:
        m1=1
        #m2=0
        m3=1
        batch=300
        
        limite=limites[veces]
        print(limites[veces])
        
        epocasInicio=2          
        model=elegirCNN(m1MP=m1,m2MP=m2,mo3D128=m3)
                    
        history=model.fit(x_train, y_train, batch_size=batch, epochs=epocasInicio, verbose=1,shuffle=True)
        score = model.evaluate(x_test, y_test, verbose = 0) 
        print(numeroEpocas)
        mejorscore=score[1]
        print(mejorscore)
                    
        while history.history['accuracy'][-1]<limite:
            numeroEpocas+=2
            history=model.fit(x_train, y_train, batch_size=300, epochs=1, verbose=1,shuffle=True)
            score = model.evaluate(x_test, y_test, verbose = 0) 
            print(f'el score interno es {score}')
            
            if score[1]>mejorscore:
                mejorscore=score[1]
                print(numeroEpocas)
                print(f'nuevo mejor score:{mejorscore} en la epoca: {numeroEpocas}')
                epocaMejorScore=numeroEpocas
                            
        
        if epocaMejorScore == 0:
            epocaMejorScore=epocasInicio
        r=(identificador,batch,numeroEpocas,m1,m2,m3,mejorscore,epocaMejorScore)
        resultados.append(r) 
        print(f'id={resultados[-1][0]}, batch={resultados[-1][1]},epoc={resultados[-1][2]},m1={resultados[-1][3]},m2={resultados[-1][4]},m3={resultados[-1][5]}, RESULTADO: {resultados[-1][6]}, epoca mejorscore={resultados[-1][0]}')         
        
        
        scoreF = model.evaluate(x_en2019,y_en2019, verbose = 0) 
        print(f'limite{limite}')
        print('Test loss:', scoreF[0]) 
        print('Test accuracy:', scoreF[1])

        
        
        if scoreF[1]>anterior:
            anterior=scoreF[1]

            #subprocess.call(["afplay","jingle.mp3"]) 
                #fiesta
            name='my_h5_modelMEJOR_PLUS_shuffled_'+str(int(anterior*100))+'.h5'
            model.save(name)
            print("******************************")
            print(scoreF[1])
                        
        identificador+=1


for x in range(len(resultados)):
    print(f'id={resultados[x][0]}, batch={resultados[x][1]},epoc={resultados[x][2]},m1={resultados[x][3]},m2={resultados[x][4]},m3={resultados[x][5]}, RESULTADO: {resultados[x][6]}, epoca mejorscore={resultados[x][7]}')         
                
        
                        

vuelta0
0.9
Epoch 1/2
12042/12042 [==============================] - 29s 2ms/step - loss: 0.6085 - accuracy: 0.6505
Epoch 2/2
12042/12042 [==============================] - 27s 2ms/step - loss: 0.3703 - accuracy: 0.8446
10
0.9124293923377991
Epoch 1/1
12042/12042 [==============================] - 26s 2ms/step - loss: 0.1817 - accuracy: 0.9316
el score interno es [0.06306016861887301, 0.9717513918876648]
12
nuevo mejor score:0.9717513918876648 en la epoca: 12
id=0, batch=300,epoc=12,m1=1,m2=0,m3=1, RESULTADO: 0.9717513918876648, epoca mejorscore=0
limite0.9
Test loss: 1.0557103946805
Test accuracy: 0.5612244606018066
vuelta1
0.96
Epoch 1/2
12042/12042 [==============================] - 27s 2ms/step - loss: 0.6254 - accuracy: 0.6460
Epoch 2/2
12042/12042 [==============================] - 26s 2ms/step - loss: 0.3740 - accuracy: 0.8391
10
0.901129961013794
Epoch 1/1
12042/12042 [==============================] - 26s 2ms/step - loss: 0.1645 - accuracy: 0.9397
el score interno es [0.089214

MEJOR RESULTADO:
id=3, batch=300,epoc=14,m1=1,m2=1,m3=1, 
RESULTADO INTERNO: 0.9830508232116699, epoca mejorscore=3
limite0.9

RESULTADO DE VALIDACION
Test loss: 0.3034899854386339
Test accuracy: 0.8734694123268127

In [24]:
#Definitiva

import subprocess

batch_size=[50,100,300] 
anterior=0.79
limites=[0.85,0.95,0.98]

resultados=[]
identificador=0
for m2 in range(2):
    for veces in range(3):
        print(f'vuelta{veces}')
        numeroEpocas=10
        epocaMejorScore=0
                    
    #blokeamos los que han dado mejor resultado:
        m1=1
        #m2=0
        m3=1
        batch=300
        
        limite=limites[veces]
        print(limites[veces])
        
        epocasInicio=2          
        model=elegirCNN(m1MP=m1,m2MP=m2,mo3D128=m3)
                    
        history=model.fit(x_train, y_train, batch_size=batch, epochs=epocasInicio, verbose=1,shuffle=True)
        score = model.evaluate(x_test, y_test, verbose = 0) 
        print(numeroEpocas)
        mejorscore=score[1]
        print(mejorscore)
                    
        while history.history['accuracy'][-1]<limite:
            numeroEpocas+=2
            history=model.fit(x_train, y_train, batch_size=300, epochs=1, verbose=1,shuffle=True)
            score = model.evaluate(x_test, y_test, verbose = 0) 
            print(f'el score interno es {score}')
            
            if score[1]>mejorscore:
                mejorscore=score[1]
                print(numeroEpocas)
                print(f'nuevo mejor score:{mejorscore} en la epoca: {numeroEpocas}')
                epocaMejorScore=numeroEpocas
                            
        
        if epocaMejorScore == 0:
            epocaMejorScore=epocasInicio
        r=(identificador,batch,numeroEpocas,m1,m2,m3,mejorscore,epocaMejorScore)
        resultados.append(r) 
        print(f'id={resultados[-1][0]}, batch={resultados[-1][1]},epoc={resultados[-1][2]},m1={resultados[-1][3]},m2={resultados[-1][4]},m3={resultados[-1][5]}, RESULTADO: {resultados[-1][6]}, epoca mejorscore={resultados[-1][0]}')         
        
        
        scoreF = model.evaluate(x_en2019,y_en2019, verbose = 0) 
        print(f'limite{limite}')
        print('Test loss:', scoreF[0]) 
        print('Test accuracy:', scoreF[1])

        
        
        if scoreF[1]>anterior:
            anterior=scoreF[1]

            #subprocess.call(["afplay","jingle.mp3"]) 
                #fiesta
            name='my_h5_modelMEJOR_PLUS_shuffled__'+str(int(anterior*100))+'.h5'
            model.save(name)
            print("******************************")
            print(scoreF[1])
            print(anterior)
                        
        identificador+=1


for x in range(len(resultados)):
    print(f'id={resultados[x][0]}, batch={resultados[x][1]},epoc={resultados[x][2]},m1={resultados[x][3]},m2={resultados[x][4]},m3={resultados[x][5]}, RESULTADO: {resultados[x][6]}, epoca mejorscore={resultados[x][7]}')         
                
        
                        

vuelta0
0.85
Epoch 1/2
12042/12042 [==============================] - 28s 2ms/step - loss: 0.6455 - accuracy: 0.6031
Epoch 2/2
12042/12042 [==============================] - 26s 2ms/step - loss: 0.4367 - accuracy: 0.7980
10
0.8559321761131287
Epoch 1/1
12042/12042 [==============================] - 27s 2ms/step - loss: 0.2023 - accuracy: 0.9192
el score interno es [0.07782661863260687, 0.9632768630981445]
12
nuevo mejor score:0.9632768630981445 en la epoca: 12
id=0, batch=300,epoc=12,m1=1,m2=0,m3=1, RESULTADO: 0.9632768630981445, epoca mejorscore=0
limite0.85
Test loss: 0.5585981173046427
Test accuracy: 0.7326530814170837
vuelta1
0.95
Epoch 1/2
12042/12042 [==============================] - 30s 2ms/step - loss: 0.6210 - accuracy: 0.6423
Epoch 2/2
12042/12042 [==============================] - 27s 2ms/step - loss: 0.3724 - accuracy: 0.8326
10
0.901129961013794
Epoch 1/1
12042/12042 [==============================] - 27s 2ms/step - loss: 0.1585 - accuracy: 0.9422
el score interno es [0.0

MEJOR RESULTADO:
id=3, batch=300,epoc=12,m1=1,m2=1,m3=1, 
RESULTADO INTERNO: 0.9237288236618042, 
epoca mejorscore=3
limite0.85

RESULTADO DE VALIDACION
Test loss: 0.192396273722454
Test accuracy: 0.9306122660636902

In [26]:
#Definitiva

import subprocess

batch_size=[50,100,300] 
anterior=0.79
limites=[0.80,0.95,0.98]

resultados=[]
identificador=0
for m3 in range(2):
    for veces in range(3):
        print(f'vuelta{veces}')
        numeroEpocas=10
        epocaMejorScore=0
                    
    #blokeamos los que han dado mejor resultado:
        m1=1
        m2=0
        #m3=1
        batch=300
        
        limite=limites[veces]
        print(limites[veces])
        
        epocasInicio=2          
        model=elegirCNN(m1MP=m1,m2MP=m2,mo3D128=m3)
                    
        history=model.fit(x_train, y_train, batch_size=batch, epochs=epocasInicio, verbose=1,shuffle=True)
        score = model.evaluate(x_test, y_test, verbose = 0) 
        print(numeroEpocas)
        mejorscore=score[1]
        print(mejorscore)
                    
        while history.history['accuracy'][-1]<limite:
            numeroEpocas+=2
            history=model.fit(x_train, y_train, batch_size=300, epochs=1, verbose=1,shuffle=True)
            score = model.evaluate(x_test, y_test, verbose = 0) 
            print(f'el score interno es {score}')
            
            if score[1]>mejorscore:
                mejorscore=score[1]
                print(numeroEpocas)
                print(f'nuevo mejor score:{mejorscore} en la epoca: {numeroEpocas}')
                epocaMejorScore=numeroEpocas
                            
        
        if epocaMejorScore == 0:
            epocaMejorScore=epocasInicio
        r=(identificador,batch,numeroEpocas,m1,m2,m3,mejorscore,epocaMejorScore)
        resultados.append(r) 
        print(f'id={resultados[-1][0]}, batch={resultados[-1][1]},epoc={resultados[-1][2]},m1={resultados[-1][3]},m2={resultados[-1][4]},m3={resultados[-1][5]}, RESULTADO: {resultados[-1][6]}, epoca mejorscore={resultados[-1][0]}')         
        
        
        scoreF = model.evaluate(x_en2019,y_en2019, verbose = 0) 
        print(f'limite{limite}')
        print('Test loss:', scoreF[0]) 
        print('Test accuracy:', scoreF[1])

        
        
        if scoreF[1]>anterior:
            anterior=scoreF[1]

            #subprocess.call(["afplay","jingle.mp3"]) 
                #fiesta
            name='my_h5_modelMEJOR_PLUS_shuffled___'+str(int(anterior*100))+'.h5'
            model.save(name)
            print("******************************")
            print(scoreF[1])
            print(anterior)
                        
        identificador+=1


for x in range(len(resultados)):
    print(f'id={resultados[x][0]}, batch={resultados[x][1]},epoc={resultados[x][2]},m1={resultados[x][3]},m2={resultados[x][4]},m3={resultados[x][5]}, RESULTADO: {resultados[x][6]}, epoca mejorscore={resultados[x][7]}')         
                
        
                        

vuelta0
0.8
Epoch 1/2
12042/12042 [==============================] - 29s 2ms/step - loss: 0.6231 - accuracy: 0.6290
Epoch 2/2
12042/12042 [==============================] - 27s 2ms/step - loss: 0.3752 - accuracy: 0.8343
10
0.9124293923377991
id=0, batch=300,epoc=10,m1=1,m2=0,m3=0, RESULTADO: 0.9124293923377991, epoca mejorscore=0
limite0.8
Test loss: 0.5096670184816633
Test accuracy: 0.8163265585899353
******************************
0.8163265585899353
0.8163265585899353
vuelta1
0.95
Epoch 1/2
12042/12042 [==============================] - 28s 2ms/step - loss: 0.6208 - accuracy: 0.6295
Epoch 2/2
12042/12042 [==============================] - 26s 2ms/step - loss: 0.3564 - accuracy: 0.8513
10
0.903954803943634
Epoch 1/1
12042/12042 [==============================] - 26s 2ms/step - loss: 0.1669 - accuracy: 0.9387
el score interno es [0.06303791699688988, 0.9830508232116699]
12
nuevo mejor score:0.9830508232116699 en la epoca: 12
Epoch 1/1
12042/12042 [==============================] - 26s 

MEJOR RESULTADO:
id=3, batch=300,
epoc=10,m1=1,m2=0,m3=1, 
RESULTADO INTERNO: 0.9322034120559692, 
epoca mejorscore=3
limite0.8

RESULTADO DE VALIDACION
Test loss: 0.17561213885795096
Test accuracy: 0.9530612230300903

In [27]:
reconstructed_model = keras.models.load_model("my_h5_modelMEJOR_PLUS_shuffled___95.h5")

score = reconstructed_model.evaluate(x_en2019, y_en2019, verbose = 0) 
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])


pred = reconstructed_model.predict(x_test) 
pred01 = np.argmax(pred, axis = 1)[0:] 
label = np.argmax(y_test,axis = 1)[0:] 

print(pred01) 
print(label)

Test loss: 0.17561213885795096
Test accuracy: 0.9530612
[1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1
 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0]
[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
